<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/DGL_learn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dgl
#import dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 17.8 MB/s eta 0:00:00


In [ ]:
import dgl

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [ ]:
import torch as th

## 第一章 图


In [ ]:
#u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
## Define edge using two groups of node IDs.
u, v = th.tensor([0,0,0,1]), th.tensor([1,2,3,4])

In [ ]:
u

tensor([0, 0, 0, 1])

In [ ]:
g = dgl.graph((u, v))

In [ ]:
print(g)

Graph(num_nodes=5, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
# Access the node IDs in the graph
print(g.nodes())

tensor([0, 1, 2, 3, 4])


In [ ]:
# Access the node ID of edges
print(g.edges())

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 4]))


In [ ]:
# Access all the node IDs of edges and the IDs of edges
print(g.edges(form='all'))

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 4]), tensor([0, 1, 2, 3]))


In [ ]:
# 如果具有最大ID的节点没有边，在创建图的时候，用户需要明确地指明节点的数量。
g = dgl.graph((u, v), num_nodes=8)

In [ ]:
print(g)

Graph(num_nodes=8, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
print(g.edges(form='all'))

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 4]), tensor([0, 1, 2, 3]))


In [ ]:
print(g.nodes()) # Now there are 8 nodes!!

tensor([0, 1, 2, 3, 4, 5, 6, 7])


In [ ]:
# 对于无向的图，用户需要为每条边都创建两个方向的边。可以使用 dgl.to_bidirected() 函数来实现这个目的。
# 如下面的代码段所示，这个函数可以把原图转换成一个包含反向边的图。
bg = dgl.to_bidirected(g)

In [ ]:
print(bg) # Now there are 8 edges

Graph(num_nodes=8, num_edges=8,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
print(bg.edges())

(tensor([0, 0, 0, 1, 1, 2, 3, 4]), tensor([1, 2, 3, 0, 4, 0, 0, 1]))


Note: 由于Tensor类内部使用C来存储，且显性定义了数据类型以及存储的设备信息，DGL推荐使用Tensor作为DGL API的输入。
DGL支持使用 32
 位或 64
 位的整数作为节点ID和边ID。节点和边ID的数据类型必须一致。如果使用 64
 位整数， DGL可以处理最多 263−1
 个节点或边。不过，如果图里的节点或者边的数量小于 231−1
 ，用户最好使用 32
 位整数。 这样不仅能提升速度，还能减少内存的使用。DGL提供了进行数据类型转换的方法，如下例所示。

In [ ]:
edges = th.tensor([2, 5, 3]), th.tensor([3, 5, 0])  # 边：2->3, 5->5, 3->0
g64 = dgl.graph(edges)
print(g64.idtype)

torch.int64


In [ ]:
g32 = dgl.graph(edges, idtype=th.int32)  # 使用int32构建图
print(g32.idtype)

torch.int32


In [ ]:
g64_2 = g32.long()  # 转换成int64
g64_2.idtype

torch.int64

In [ ]:
g32_2 = g64.int()  # 转换成int32
g32_2.idtype

torch.int32

Create attributes for nodes and edges of Graph

In [ ]:
print(g.num_nodes())

8


In [ ]:
g.ndata['x'] = th.ones(g.num_nodes(), 3) # each node have a vector [1,1,1] and the attribute name is 'x'

In [ ]:
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # each edge has a scalar 1 and integer type, and the attribute name is 'x'

In [ ]:
print(g)

Graph(num_nodes=8, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})


In [ ]:
g.ndata['x'][2] #  # 获取节点2的特征

tensor([1., 1., 1.])

In [ ]:
g.ndata['y'] = th.randn(g.num_nodes(), 5) ## Add a new group of attributes named 'y' for nodes.

In [ ]:
g.edata['x'][th.tensor([0, 3])]  # 获取边0和3的特征

tensor([1, 1], dtype=torch.int32)

关于 ndata 和 edata 接口的重要说明：仅允许使用数值类型（如单精度浮点型、双精度浮点型和整型）的特征。这些特征可以是标量、向量或多维张量。

每个节点特征具有唯一名称，每个边特征也具有唯一名称。节点和边的特征可以具有相同的名称（如上述示例代码中的 'x' ）。

通过张量分配创建特征时，DGL会将特征赋给图中的每个节点和每条边。该张量的第一维必须与图中节点或边的数量一致。 不能将特征赋给图中节点或边的子集。

相同名称的特征必须具有相同的维度和数据类型。

特征张量使用”行优先”的原则，即每个行切片储存1个节点或1条边的特征（参考上述示例代码的第16和18行）。

对于加权图，用户可以将权重储存为一个边特征，如下。

In [ ]:
# 边 0->1, 0->2, 0->3, 1->3
edges = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
weights = th.tensor([0.1, 0.6, 0.9, 0.7])  # 每条边的权重
g = dgl.graph(edges)
g.edata['w'] = weights  # 将其命名为 'w'
print(g)


Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})


1.4 从外部源创建图¶

In [ ]:
#可以从外部来源构造一个 DGLGraph 对象，包括：
# 1.从用于图和稀疏矩阵的外部Python库（NetworkX 和 SciPy）创建而来。
# 2. 从磁盘加载图数据。
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5%非零项
g2 = dgl.from_scipy(spmat)
print(g2)

Graph(num_nodes=100, num_edges=500,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
print(g2.nodes())

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


In [ ]:
print(g2.edges(form='all'))

(tensor([45,  7, 13, 10, 25, 16, 95, 99, 71, 78, 97, 53, 42, 47, 15, 16, 23, 32,
        48, 32, 31, 83, 58, 25, 42, 67, 22, 44, 31, 24,  9, 14, 69, 37, 66, 39,
         2, 34, 35, 90, 74,  5, 45, 31, 26, 80, 96, 55, 82, 39, 11, 17, 14, 62,
        52, 44, 16, 38, 96,  9, 72, 33, 49, 99, 14,  2, 89, 12, 15,  3, 84, 66,
        28, 67, 92, 38, 71, 56, 86, 27, 93, 95, 22, 25, 44, 67,  5, 70, 69, 21,
        32, 21, 10, 52, 99, 25, 68, 61, 44, 25, 29, 15, 64, 20, 69, 52, 77, 20,
        60, 94, 24, 75, 38, 94,  6, 67, 46, 14, 43, 53, 79, 88, 17, 74, 50, 64,
        73,  3, 75, 79, 30, 53, 12, 13, 39, 62, 66, 41, 34, 12, 20, 65, 89, 28,
        38,  9, 82, 85, 53, 11, 67, 95, 19, 22, 23, 71,  4, 27, 34, 36,  6, 16,
        93, 79, 82, 75, 91, 39, 12, 70, 40, 86, 24, 42, 70, 38, 50, 47, 60, 16,
        48, 30, 96, 82, 24, 12, 93, 96,  0, 81, 74, 10, 48, 36, 58, 27, 56, 19,
        45, 43,  5, 88, 97,  5, 20,  4, 17, 88, 27, 56, 41, 21, 47, 12, 23, 13,
        56, 24, 84, 87, 43, 32,  5,  7,

In [ ]:
import networkx as nx
nx_g = nx.path_graph(5) # 一条链路0-1-2-3-4
dgl.from_networkx(nx_g) #

Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

Note: 注意，当使用 nx.path_graph(5) 进行创建时， DGLGraph 对象有8条边，而非4条。 这是由于 nx.path_graph(5) 构建了一个无向的NetworkX图 networkx.Graph ，而 DGLGraph 的边总是有向的。 所以当将无向的NetworkX图转换为 DGLGraph 对象时，DGL会在内部将1条无向边转换为2条有向边。 使用有向的NetworkX图 networkx.DiGraph 可避免该行为。

In [ ]:
nxg = nx.DiGraph([(2, 1), (1, 2), (2, 3), (0, 0)])
dgl.from_networkx(nxg)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

从磁盘加载图¶
有多种文件格式可储存图，所以这里难以枚举所有选项。本节仅给出一些常见格式的一般情况。

逗号分隔值（CSV）.CSV是一种常见的格式，以表格格式储存节点、边及其特征：

1.5 创建异构图

In [ ]:
import dgl
import torch as th

# 创建一个具有3种节点类型和3种边类型的异构图
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
g.ntypes

['disease', 'drug', 'gene']

In [ ]:
print(g)

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])


In [ ]:
print(g.etypes) # print edge types

['interacts', 'interacts', 'treats']


In [ ]:
print(g.canonical_etypes) # output the canonical edge types.

[('drug', 'interacts', 'drug'), ('drug', 'interacts', 'gene'), ('drug', 'treats', 'disease')]


Note: 注意，同构图和二分图只是一种特殊的异构图，它们只包括一种关系。

In [ ]:
# 一个同构图
dgl.heterograph({('node_type', 'edge_type', 'node_type'): (u, v)})


Graph(num_nodes=5, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
# 一个二分图
dgl.heterograph({('source_type', 'edge_type', 'destination_type'): (u, v)})

Graph(num_nodes={'destination_type': 5, 'source_type': 2},
      num_edges={('source_type', 'edge_type', 'destination_type'): 4},
      metagraph=[('source_type', 'destination_type', 'edge_type')])

In [ ]:
print(g)

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])


In [ ]:
print(g.metagraph().edges())

[('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')]


In [ ]:
# 当引入多种节点和边类型后，用户在调用DGLGraph API以获取特定类型的信息时，需要指定具体的节点和边类型。此外，不同类型的节点和边具有单独的ID。
# 获取图中所有节点的数量
g.num_nodes()

10

In [ ]:
# 获取drug节点的数量
g.num_nodes('drug')

3

In [ ]:
# 不同类型的节点有单独的ID。因此，没有指定节点类型就没有明确的返回值。
# g.nodes() will produce error

In [ ]:
g.nodes('drug')

tensor([0, 1, 2])

In [ ]:
# 为了设置/获取特定节点和边类型的特征，DGL提供了两种新类型的语法：
# g.nodes[‘node_type’].data[‘feat_name’] 和 g.edges[‘edge_type’].data[‘feat_name’] 。
# 设置/获取"drug"类型的节点的"hv"特征
g.nodes['drug'].data['hv'] = th.ones(3, 1)
g.nodes['drug'].data['hv']

tensor([[1.],
        [1.],
        [1.]])

In [ ]:
# 设置/获取"treats"类型的边的"he"特征
g.edges['treats'].data['he'] = th.zeros(1, 1)
g.edges['treats'].data['he']

tensor([[0.]])

In [ ]:
# 如果图里只有一种节点或边类型，则不需要指定节点或边的类型。
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'is similar', 'drug'): (th.tensor([0, 1]), th.tensor([2, 3]))
})
g.nodes()

tensor([0, 1, 2, 3])

In [ ]:
g.ndata['hv'] = th.ones(4, 1)

Note: 当边类型唯一地确定了源节点和目标节点的类型时，用户可以只使用一个字符串而不是字符串三元组来指定边类型。例如， 对于具有两个关系 ('user', 'plays', 'game') 和 ('user', 'likes', 'game') 的异构图， 只使用 'plays' 或 'like' 来指代这两个关系是可以的。

逗号分隔值（CSV）¶
一种存储异构图的常见方法是在不同的CSV文件中存储不同类型的节点和边。下面是一个例子。


In [ ]:
# 数据文件夹
data/
|-- drug.csv        # drug节点
|-- gene.csv        # gene节点
|-- disease.csv     # disease节点
|-- drug-interact-drug.csv  # drug-drug相互作用边
|-- drug-interact-gene.csv  # drug-gene相互作用边
|-- drug-treat-disease.csv  # drug-disease治疗边

In [ ]:
# 边类型子图
# 用户可以通过指定要保留的关系来创建异构图的子图，相关的特征也会被拷贝。
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})
g.nodes['drug'].data['hv'] = th.ones(3, 1)

# 保留关系 ('drug', 'interacts', 'drug') 和 ('drug', 'treats', 'disease') 。
# 'drug' 和 'disease' 类型的节点也会被保留
eg = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                ('drug', 'treats', 'disease')])
eg

Graph(num_nodes={'disease': 3, 'drug': 3},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'disease', 'treats')])

In [ ]:
eg.nodes['drug'].data['hv']

tensor([[1.],
        [1.],
        [1.]])

In [ ]:
# 出于建模的目的，用户可能需要将一些关系合并，并对它们应用相同的操作。为了实现这一目的，可以先抽取异构图的边类型子图，然后将该子图转换为同构图。
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})
print(g)

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])


In [ ]:
sub_g = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                   ('drug', 'interacts', 'gene')])
h_sub_g = dgl.to_homogeneous(sub_g)
print(h_sub_g)

Graph(num_nodes=7, num_edges=4,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})


In [ ]:
h_sub_g.nodes()

tensor([0, 1, 2, 3, 4, 5, 6])

In [ ]:
h_sub_g.edges()

(tensor([0, 1, 0, 1]), tensor([1, 2, 5, 6]))

In [ ]:
h_sub_g.ndata

{'_ID': tensor([0, 1, 2, 0, 1, 2, 3]), '_TYPE': tensor([0, 0, 0, 1, 1, 1, 1])}

In [ ]:
h_sub_g.edata

{'_ID': tensor([0, 1, 0, 1]), '_TYPE': tensor([0, 0, 1, 1])}

1.6 在GPU上使用DGLGraph¶

In [ ]:
# 用户可以通过在构造过程中传入两个GPU张量来创建GPU上的 DGLGraph 。
# 另一种方法是使用 to() API将 DGLGraph 复制到GPU，这会将图结构和特征数据都拷贝到指定的设备。
import dgl
import torch as th
u, v = th.tensor([0, 1, 2]), th.tensor([2, 3, 4])
g = dgl.graph((u, v))
g.ndata['x'] = th.randn(5, 3)   # 原始特征在CPU上
g.device

device(type='cpu')

In [ ]:
if th.cuda.is_available():
    device = th.device('cuda')
else:
    device = th.device('cpu')

print('Using device:', device)

Using device: cuda


In [ ]:
#!pip install dgl-cu102

ERROR: Could not find a version that satisfies the requirement dgl-cu102 (from versions: none)
ERROR: No matching distribution found for dgl-cu102


In [ ]:
## the colab's cuda is called "cuda:0"
cuda_g = g.to('cuda:0')         # 接受来自后端框架的任何设备对象
cuda_g.device

device(type='cuda', index=0)

In [ ]:
# 由GPU张量构造的图也在GPU上
u, v = u.to('cuda:0'), v.to('cuda:0')
g = dgl.graph((u, v))
g.device

device(type='cuda', index=0)

In [ ]:
# 任何涉及GPU图的操作都是在GPU上运行的。因此，这要求所有张量参数都已经放在GPU上，其结果(图或张量)也将在GPU上。
# 此外，GPU图只接受GPU上的特征数据。
cuda_g


Graph(num_nodes=5, num_edges=3,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
# g.in_degrees()
g

Graph(num_nodes=5, num_edges=3,
      ndata_schemes={}
      edata_schemes={})

In [ ]:
#cuda_g.in_edges([2, 3, 4])
# cuda_g.in_edges(th.tensor([2, 3, 4]).to('cuda:0'))  #

## 第5章：训练图神经网络

### 5.1 节点分类/回归

In [2]:
import dgl

dataset = dgl.data.CiteseerGraphDataset()
graph = dataset[0]

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/citeseer_d6836239
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [3]:
print(graph)

Graph(num_nodes=3327, num_edges=9228,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(3703,), dtype=torch.float32)}
      edata_schemes={})


In [ ]:
type(graph)

dgl.heterograph.DGLGraph

In [ ]:
import numpy as np
print(np.random.randint(0, 10, 100)) ## generate 100  integers ranged [0-9]

[0 1 5 4 0 7 1 9 9 9 2 0 8 1 1 1 1 2 8 9 5 8 5 8 9 9 7 4 1 1 0 0 5 4 6 1 8
 5 0 6 9 2 3 6 9 5 1 3 7 6 1 5 8 7 5 2 2 5 7 5 3 4 5 5 2 8 4 8 1 5 7 4 3 2
 0 5 0 9 2 9 8 9 0 0 8 4 2 6 4 8 3 4 8 9 4 8 5 9 1 6]


In [ ]:
print(hetero_graph)

Graph(num_nodes={'item': 500, 'user': 1000},
      num_edges={('item', 'clicked-by', 'user'): 5000, ('item', 'disliked-by', 'user'): 500, ('user', 'click', 'item'): 5000, ('user', 'dislike', 'item'): 500, ('user', 'follow', 'user'): 3000, ('user', 'followed-by', 'user'): 3000},
      metagraph=[('item', 'user', 'clicked-by'), ('item', 'user', 'disliked-by'), ('user', 'item', 'click'), ('user', 'item', 'dislike'), ('user', 'user', 'follow'), ('user', 'user', 'followed-by')])


In [4]:
# 构建一个2层的GNN模型
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        # 实例化SAGEConv，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregator_type是聚合函数的类型
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # 输入是节点的特征
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [5]:
SAGE

__main__.SAGE

In [ ]:
print(graph)

Graph(num_nodes=3327, num_edges=9228,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(3703,), dtype=torch.float32)}
      edata_schemes={})


In [6]:
node_features = graph.ndata['feat']
node_labels = graph.ndata['label']
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [7]:
def evaluate(model, graph, features, labels, mask):
    model.eval() # Sets the model to evaluation mode. In PyTorch, this is important because it disables operations like dropout,
    # which are usually active during training but should be turned off during evaluation.
    # This context manager ensures that during the following block of code, PyTorch won't track operations for gradient computation.
    # This is done for efficiency during evaluation since gradients are not needed.
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [8]:
import torch

In [9]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(500):
  # Puts the model in training mode. This is necessary because some layers,
  # like dropout or batch normalization, behave differently during training compared to evaluation.
    model.train() # set model to training mode: compute the grad autoly for parameters
    # 使用所有节点(全图)进行前向传播计算
    logits = model(graph, node_features)
    # 计算损失值
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # 计算验证集的准确度
    acc = evaluate(model, graph, node_features, node_labels, valid_mask)
    # 进行反向传播计算
    # Clears the gradients of all optimized tensors. This is necessary before computing the gradients for the next minibatch.
    opt.zero_grad()
    # Backward pass to compute the gradients of the loss with respect to the model parameters.
    loss.backward()
    # Updates the model parameters based on the computed gradients using the optimization algorithm (assumed to be stored in the opt variable).
    opt.step()
    # Prints the value of the training loss for the current epoch.
    if(epoch % 50 ==0):
      print(loss.item())

    # 如果需要的话，保存训练好的模型。本例中省略。

1.792499303817749
0.7453944683074951
0.16209319233894348
0.05643106997013092
0.02895241416990757
0.01785055361688137
0.012196316383779049
0.008899891749024391
0.006799372378736734
0.005373336840420961


In [10]:
print(loss) #
print(loss.item())
print(evaluate(model, graph, node_features, node_labels, valid_mask))

tensor(0.0044, grad_fn=<NllLossBackward0>)
0.004375432152301073
0.662


In [11]:
## Test
evaluate(model, graph, node_features, node_labels, test_mask)

0.657

异构图上的节点分类模型的训练

In [12]:
# How to create a heter graph
import dgl
import numpy as np
import torch

n_users = 1000
n_items = 500
## two nodes type
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
## three relation types
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
# 在user类型的节点和click类型的边上随机生成训练集的掩码
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)


In [13]:
# Define a Heterograph Conv model

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()
        # 实例化HeteroGraphConv，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregate是聚合函数的类型
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # 输入是节点的特征字典
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [14]:
print(hetero_graph.etypes)

['clicked-by', 'disliked-by', 'click', 'dislike', 'follow', 'followed-by']


In [16]:
model = RGCN(n_hetero_features, 20, n_user_classes, hetero_graph.etypes)
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
labels = hetero_graph.nodes['user'].data['label']
train_mask = hetero_graph.nodes['user'].data['train_mask']

In [19]:
# 用户可以简单地按如下形式进行前向传播计算：
node_features = {'user': user_feats, 'item': item_feats}
h_dict = model(hetero_graph, node_features)
h_user = h_dict['user']
h_item = h_dict['item']
print(h_item)
print(h_user)

tensor([[-0.0575,  0.6067, -0.0424,  0.2786,  1.0782],
        [-0.5531, -0.2752,  0.1740, -0.3414,  0.1866],
        [-0.1681, -0.1661, -0.0463,  0.0045, -0.1572],
        ...,
        [-0.5999, -0.0726, -0.1106, -0.2612,  0.2400],
        [-0.0139,  0.7908, -1.2675, -0.8072,  0.3811],
        [-0.1872, -0.1866, -0.3842, -0.0780, -0.4704]], grad_fn=<SumBackward1>)
tensor([[-0.6181,  1.3472,  0.8948, -0.1486,  0.9954],
        [ 0.2136, -0.2236,  1.4053,  1.5777, -0.1048],
        [-0.4757,  0.9316,  2.4770,  0.8157,  1.1059],
        ...,
        [-0.5110,  1.3694, -0.3583,  1.0716,  0.5826],
        [-0.1496,  1.4201, -0.3755,  0.7165,  0.1973],
        [-0.7551,  0.8209, -0.0413,  0.9648,  0.1524]], grad_fn=<SumBackward1>)


In [20]:
# 异构图上模型的训练和同构图的模型训练是一样的，只是这里使用了一个包括节点表示的字典来计算预测值。
# 例如，如果只预测 user 节点的类别，用户可以从返回的字典中提取 user 的节点嵌入。
opt = torch.optim.Adam(model.parameters())

for epoch in range(5):
    model.train()
    # 使用所有节点的特征进行前向传播计算，并提取输出的user节点嵌入
    logits = model(hetero_graph, node_features)['user']
    # 计算损失值
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    # 计算验证集的准确度。在本例中省略。
    # 进行反向传播计算
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(f'The traning loss is: {loss.item(): 4.0f}')


The traning loss is:    2
The traning loss is:    2
The traning loss is:    2
The traning loss is:    2
The traning loss is:    2


### 5.3 链接预测

In [25]:
import torch.nn as nn
import dgl.function as fn
class DotProductPredictor(nn.Module):
    def forward(self, graph, h):
        # h是从5.1节的GNN模型中计算出的节点表示
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return graph.edata['score']

In [22]:
def construct_negative_graph(graph, k):
    src, dst = graph.edges()

    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.num_nodes(), (len(src) * k,))
    return dgl.graph((neg_src, neg_dst), num_nodes=graph.num_nodes())

In [23]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.sage = SAGE(in_features, hidden_features, out_features)
        self.pred = DotProductPredictor()
    def forward(self, g, neg_g, x):
        h = self.sage(g, x)
        return self.pred(g, h), self.pred(neg_g, h)

In [26]:
def compute_loss(pos_score, neg_score):
    # 间隔损失
    n_edges = pos_score.shape[0]
    return (1 - pos_score.unsqueeze(1) + neg_score.view(n_edges, -1)).clamp(min=0).mean()

node_features = graph.ndata['feat']
n_features = node_features.shape[1]
k = 5
model = Model(n_features, 100, 100)
opt = torch.optim.Adam(model.parameters())
for epoch in range(10):
    negative_graph = construct_negative_graph(graph, k)
    pos_score, neg_score = model(graph, negative_graph, node_features)
    loss = compute_loss(pos_score, neg_score)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

0.998602032661438
0.9959600567817688
0.9929999709129333
0.9893460273742676
0.9846848845481873
0.9791660904884338
0.9718270897865295
0.9635816812515259
0.952874481678009
0.9406253695487976


In [ ]:
node_embeddings = model.sage(graph, node_features)

## 第6章 大图随机批次训练

### 6.1 针对节点分类任务的邻居采样训练方法

In [ ]:
!pip show dgl

Name: dgl
Version: 1.1.3
Summary: Deep Graph Library
Home-page: https://github.com/dmlc/dgl
Author: 
Author-email: 
License: APACHE
Location: /usr/local/lib/python3.10/dist-packages
Requires: networkx, numpy, psutil, requests, scipy, tqdm
Required-by: 


In [ ]:
import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
g = graph
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
#dataloader = dgl.dataloading.DataLoader(g, train_nid_dict, sampler,
#    batch_size=1024,shuffle=True,drop_last=False,num_workers=4)

In [31]:
# 调整模型以进行小批次训练
class TwoLayerGCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.conv1 = dglnn.GraphConv(in_features, hidden_features)
        self.conv2 = dglnn.GraphConv(hidden_features, out_features)

    def forward(self, g, x):
        x = F.relu(self.conv1(g, x))
        x = F.relu(self.conv2(g, x))
        return x

In [32]:
# 然后，用户所需要做的就是用上面生成的块( block )来替换图( g )。
class StochasticTwoLayerGCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.conv1 = dgl.nn.GraphConv(in_features, hidden_features)
        self.conv2 = dgl.nn.GraphConv(hidden_features, out_features)

    def forward(self, blocks, x):
        x = F.relu(self.conv1(blocks[0], x))
        x = F.relu(self.conv2(blocks[1], x))
        return x

In [ ]:
model = StochasticTwoLayerGCN(in_features, hidden_features, out_features)
model = model.cuda()
opt = torch.optim.Adam(model.parameters())

for input_nodes, output_nodes, blocks in dataloader:
    blocks = [b.to(torch.device('cuda')) for b in blocks]
    input_features = blocks[0].srcdata['features']
    output_labels = blocks[-1].dstdata['label']
    output_predictions = model(blocks, input_features)
    loss = compute_loss(output_labels, output_predictions)
    opt.zero_grad()
    loss.backward()
    opt.step()

完整示例

In [33]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 7.7 MB/s eta 0:00:00


In [35]:
!pip install ogb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=d4022d201bc54f776dc401956b915fcbd91310b8f04fe8d0a2b095ccf2406d06
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [36]:
import argparse

import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as MF
import tqdm
from dgl.data import AsNodePredDataset
from dgl.dataloading import (
    DataLoader,
    MultiLayerFullNeighborSampler,
    NeighborSampler,
)
from ogb.nodeproppred import DglNodePropPredDataset

In [47]:
class SAGE(nn.Module):
    def __init__(self, in_size, hid_size, out_size):
        super().__init__()
        self.layers = nn.ModuleList()
        # three-layer GraphSAGE-mean
        self.layers.append(dglnn.SAGEConv(in_size, hid_size, "mean"))
        self.layers.append(dglnn.SAGEConv(hid_size, hid_size, "mean"))
        self.layers.append(dglnn.SAGEConv(hid_size, out_size, "mean"))
        self.dropout = nn.Dropout(0.5)
        self.hid_size = hid_size
        self.out_size = out_size

    def forward(self, blocks, x):
        h = x
        for l, (layer, block) in enumerate(zip(self.layers, blocks)):
            h = layer(block, h)
            if l != len(self.layers) - 1:
                h = F.relu(h)
                h = self.dropout(h)
        return h

    def inference(self, g, device, batch_size):
        """Conduct layer-wise inference to get all the node embeddings."""
        feat = g.ndata["feat"]
        sampler = MultiLayerFullNeighborSampler(1, prefetch_node_feats=["feat"])
        dataloader = DataLoader(
            g,
            torch.arange(g.num_nodes()).to(g.device),
            sampler,
            device=device,
            batch_size=batch_size,
            shuffle=False,
            drop_last=False,
            num_workers=0,
        )
        buffer_device = torch.device("cpu")
        pin_memory = buffer_device != device

        for l, layer in enumerate(self.layers):
            y = torch.empty(
                g.num_nodes(),
                self.hid_size if l != len(self.layers) - 1 else self.out_size,
                dtype=feat.dtype,
                device=buffer_device,
                pin_memory=pin_memory,
            )
            feat = feat.to(device)
            for input_nodes, output_nodes, blocks in tqdm.tqdm(dataloader):
                x = feat[input_nodes]
                h = layer(blocks[0], x)  # len(blocks) = 1
                if l != len(self.layers) - 1:
                    h = F.relu(h)
                    h = self.dropout(h)
                # by design, our output nodes are contiguous
                y[output_nodes[0] : output_nodes[-1] + 1] = h.to(buffer_device)
            feat = y
        return y


In [46]:
def evaluate(model, graph, dataloader, num_classes):
    model.eval()
    ys = []
    y_hats = []
    for it, (input_nodes, output_nodes, blocks) in enumerate(dataloader):
        with torch.no_grad():
            x = blocks[0].srcdata["feat"]
            ys.append(blocks[-1].dstdata["label"])
            y_hats.append(model(blocks, x))
    return MF.accuracy(
        torch.cat(y_hats),
        torch.cat(ys),
        task="multiclass",
        num_classes=num_classes,
    )


In [45]:
def layerwise_infer(device, graph, nid, model, num_classes, batch_size):
    model.eval()
    with torch.no_grad():
        pred = model.inference(
            graph, device, batch_size
        )  # pred in buffer_device
        pred = pred[nid]
        label = graph.ndata["label"][nid].to(pred.device)
        return MF.accuracy(
            pred, label, task="multiclass", num_classes=num_classes
        )


In [49]:
def train(mode, device, g, dataset, model, num_classes):
    # create sampler & dataloader
    train_idx = dataset.train_idx.to(device)
    val_idx = dataset.val_idx.to(device)
    sampler = NeighborSampler(
        [10, 10, 10],  # fanout for [layer-0, layer-1, layer-2]
        prefetch_node_feats=["feat"],
        prefetch_labels=["label"],
    )
    use_uva = mode == "mixed"
    train_dataloader = DataLoader(
        g,
        train_idx,
        sampler,
        device=device,
        batch_size=1024,
        shuffle=True,
        drop_last=False,
        num_workers=0,
        use_uva=use_uva,
    )

    val_dataloader = DataLoader(
        g,
        val_idx,
        sampler,
        device=device,
        batch_size=1024,
        shuffle=True,
        drop_last=False,
        num_workers=0,
        use_uva=use_uva,
    )

    opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

    for epoch in range(10):
        model.train()
        total_loss = 0
        for it, (input_nodes, output_nodes, blocks) in enumerate(
            train_dataloader
        ):
            x = blocks[0].srcdata["feat"]
            y = blocks[-1].dstdata["label"]
            y_hat = model(blocks, x)
            loss = F.cross_entropy(y_hat, y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        acc = evaluate(model, g, val_dataloader, num_classes)
        print(
            "Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} ".format(
                epoch, total_loss / (it + 1), acc.item()
            )
        )


In [41]:
args = {'mode': 'mixed'}
print(args['mode'])

mixed


In [43]:
mode = 'mixed'
dt = "bfloat16"
if not torch.cuda.is_available():
        mode = "cpu"
print(f"Training in {mode} mode.")

# load and preprocess dataset
print("Loading data")
dataset = AsNodePredDataset(DglNodePropPredDataset("ogbn-products"))



Training in mixed mode.
Loading data
This will download 1.38GB. Will you proceed? (y/N)
y


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [00:20<00:00, 70.00it/s]


Extracting dataset/products.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:02<00:00,  2.13s/it]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  2.93it/s]


Saving...
Training...


NameError: ignored

In [55]:
mode = 'mixed'
g = dataset[0]
g = g.to("cuda" if mode == "puregpu" else "cpu")
num_classes = dataset.num_classes
device = torch.device("cpu" if mode == "cpu" else "cuda")

    # create GraphSAGE model
in_size = g.ndata["feat"].shape[1]
out_size = dataset.num_classes
model = SAGE(in_size, 256, out_size).to(device)

    # convert model and graph to bfloat16 if needed
if dt == "bfloat16":
        g = dgl.to_bfloat16(g)
        model = model.to(dtype=torch.bfloat16)

    # model training
print(g)

Graph(num_nodes=2449029, num_edges=123718280,
      ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


In [53]:
print("Training...")
train(mode, device, g, dataset, model, num_classes)
   # test the model
print("Testing...")
acc = layerwise_infer(
        device, g, dataset.test_idx, model, num_classes, batch_size=4096
    )
print("Test Accuracy {:.4f}".format(acc.item()))

Training...


/usr/local/lib/python3.10/dist-packages/dgl/dataloading/dataloader.py:1149: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(


KeyboardInterrupt: ignored

In [37]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--mode",
        default="mixed",
        choices=["cpu", "mixed", "puregpu"],
        help="Training mode. 'cpu' for CPU training, 'mixed' for CPU-GPU mixed training, "
        "'puregpu' for pure-GPU training.",
    )
    parser.add_argument(
        "--dt",
        type=str,
        default="float",
        help="data type(float, bfloat16)",
    )
    args = parser.parse_args()
    if not torch.cuda.is_available():
        args.mode = "cpu"
    print(f"Training in {args.mode} mode.")

    # load and preprocess dataset
    print("Loading data")
    dataset = AsNodePredDataset(DglNodePropPredDataset("ogbn-products"))
    g = dataset[0]
    g = g.to("cuda" if args.mode == "puregpu" else "cpu")
    num_classes = dataset.num_classes
    device = torch.device("cpu" if args.mode == "cpu" else "cuda")

    # create GraphSAGE model
    in_size = g.ndata["feat"].shape[1]
    out_size = dataset.num_classes
    model = SAGE(in_size, 256, out_size).to(device)

    # convert model and graph to bfloat16 if needed
    if args.dt == "bfloat16":
        g = dgl.to_bfloat16(g)
        model = model.to(dtype=torch.bfloat16)

    # model training
    print("Training...")
    train(args, device, g, dataset, model, num_classes)

    # test the model
    print("Testing...")
    acc = layerwise_infer(
        device, g, dataset.test_idx, model, num_classes, batch_size=4096
    )
    print("Test Accuracy {:.4f}".format(acc.item()))

usage: colab_kernel_launcher.py [-h] [--mode {cpu,mixed,puregpu}] [--dt DT]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-62d4f8b3-c678-4378-9fe4-e9c910ea3d22.json


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 6.1.1 异构图上的训练


In [ ]:
# 在异构图上训练图神经网络进行节点分类的方法也是类似的。
class StochasticTwoLayerRGCN(nn.Module):
    def __init__(self, in_feat, hidden_feat, out_feat, rel_names):
        super().__init__()
        self.conv1 = dglnn.HeteroGraphConv({
                rel : dglnn.GraphConv(in_feat, hidden_feat, norm='right')
                for rel in rel_names
            })
        self.conv2 = dglnn.HeteroGraphConv({
                rel : dglnn.GraphConv(hidden_feat, out_feat, norm='right')
                for rel in rel_names
            })

    def forward(self, blocks, x):
        x = self.conv1(blocks[0], x)
        x = self.conv2(blocks[1], x)
        return x


In [ ]:
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
#dataloader = dgl.dataloading.DataLoader(g, train_nid_dict, sampler,
#    batch_size=1024,shuffle=True,drop_last=False,num_workers=4)

In [ ]:
# 模型的训练与同构图几乎相同。不同之处在于， compute_loss 的实现会包含两个字典：节点类型和预测结果。
model = StochasticTwoLayerRGCN(in_features, hidden_features, out_features, etypes)
model = model.cuda()
opt = torch.optim.Adam(model.parameters())

for input_nodes, output_nodes, blocks in dataloader:
    blocks = [b.to(torch.device('cuda')) for b in blocks]
    input_features = blocks[0].srcdata     # returns a dict
    output_labels = blocks[-1].dstdata     # returns a dict
    output_predictions = model(blocks, input_features)
    loss = compute_loss(output_labels, output_predictions)
    opt.zero_grad()
    loss.backward()
    opt.step()

In [62]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.8 MB/s eta 0:00:00


实现的案例

In [58]:
"""Modeling Relational Data with Graph Convolutional Networks
Paper: https://arxiv.org/abs/1703.06103
Reference Code: https://github.com/tkipf/relational-gcn
"""
import argparse
import itertools
import time

import dgl

import numpy as np
import torch as th
import torch.nn.functional as F
from dgl.data.rdf import AIFBDataset, AMDataset, BGSDataset, MUTAGDataset
#from model import EntityClassify, RelGraphEmbed

In [59]:

def extract_embed(node_embed, input_nodes):
    emb = {}
    for ntype, nid in input_nodes.items():
        nid = input_nodes[ntype]
        emb[ntype] = node_embed[ntype][nid]
    return emb


In [60]:

def evaluate(model, loader, node_embed, labels, category, device):
    model.eval()
    total_loss = 0
    total_acc = 0
    count = 0
    with loader.enable_cpu_affinity():
        for input_nodes, seeds, blocks in loader:
            blocks = [blk.to(device) for blk in blocks]
            seeds = seeds[category]
            emb = extract_embed(node_embed, input_nodes)
            emb = {k: e.to(device) for k, e in emb.items()}
            lbl = labels[seeds].to(device)
            logits = model(emb, blocks)[category]
            loss = F.cross_entropy(logits, lbl)
            acc = th.sum(logits.argmax(dim=1) == lbl).item()
            total_loss += loss.item() * len(seeds)
            total_acc += acc
            count += len(seeds)
    return total_loss / count, total_acc / count



In [63]:
dataset = AIFBDataset()

Extracting file to /root/.dgl/aifb-hetero_82d021d8
Parsing file aifbfixed_complete.n3 ...
Processed 0 tuples, found 0 valid tuples.
Processed 10000 tuples, found 8406 valid tuples.
Processed 20000 tuples, found 16622 valid tuples.
Adding reverse edges ...
Creating one whole graph ...
Total #nodes: 7262
Total #edges: 48810
Convert to heterograph ...
#Node types: 7
#Canonical edge types: 104
#Unique edge type names: 78
Load training/validation/testing split ...
Done saving data into cached files.


In [66]:
dataset
g = dataset[0]
category = dataset.predict_category
num_classes = dataset.num_classes
train_mask = g.nodes[category].data.pop("train_mask")
test_mask = g.nodes[category].data.pop("test_mask")
train_idx = th.nonzero(train_mask, as_tuple=False).squeeze()
test_idx = th.nonzero(test_mask, as_tuple=False).squeeze()
labels = g.nodes[category].data.pop("labels")

In [67]:
print(g)
val_idx = train_idx[: len(train_idx) // 5]
train_idx = train_idx[len(train_idx) // 5 :]
embed_layer = RelGraphEmbed(g, args.n_hidden)
labels = labels.to(device)
embed_layer = embed_layer.to(device)

Graph(num_nodes={'Forschungsgebiete': 146, 'Forschungsgruppen': 5, 'Kooperationen': 28, 'Personen': 237, 'Projekte': 78, 'Publikationen': 1318, '_Literal': 5450},
      num_edges={('Forschungsgebiete', 'ontology#dealtWithIn', 'Projekte'): 357, ('Forschungsgebiete', 'ontology#isWorkedOnBy', 'Personen'): 571, ('Forschungsgebiete', 'ontology#name', '_Literal'): 146, ('Forschungsgebiete', 'rdftype', '_Literal'): 129, ('Forschungsgebiete', 'rev-ontology#isAbout', 'Projekte'): 357, ('Forschungsgebiete', 'rev-ontology#isAbout', 'Publikationen'): 2120, ('Forschungsgruppen', 'ontology#carriesOut', 'Projekte'): 79, ('Forschungsgruppen', 'ontology#head', 'Personen'): 5, ('Forschungsgruppen', 'ontology#homepage', '_Literal'): 5, ('Forschungsgruppen', 'ontology#member', 'Personen'): 74, ('Forschungsgruppen', 'ontology#name', '_Literal'): 5, ('Forschungsgruppen', 'ontology#publishes', 'Publikationen'): 1217, ('Forschungsgruppen', 'rev-ontology#carriedOutBy', 'Projekte'): 79, ('Kooperationen', 'ontol

In [ ]:
def main(args):
    # check cuda
    device = "cpu"
    use_cuda = args.gpu >= 0 and th.cuda.is_available()
    if use_cuda:
        th.cuda.set_device(args.gpu)
        device = "cuda:%d" % args.gpu

    # load graph data
    if args.dataset == "aifb":
        dataset = AIFBDataset()
    elif args.dataset == "mutag":
        dataset = MUTAGDataset()
    elif args.dataset == "bgs":
        dataset = BGSDataset()
    elif args.dataset == "am":
        dataset = AMDataset()
    else:
        raise ValueError()

    g = dataset[0]
    category = dataset.predict_category
    num_classes = dataset.num_classes
    train_mask = g.nodes[category].data.pop("train_mask")
    test_mask = g.nodes[category].data.pop("test_mask")
    train_idx = th.nonzero(train_mask, as_tuple=False).squeeze()
    test_idx = th.nonzero(test_mask, as_tuple=False).squeeze()
    labels = g.nodes[category].data.pop("labels")

    # split dataset into train, validate, test
    if args.validation:
        val_idx = train_idx[: len(train_idx) // 5]
        train_idx = train_idx[len(train_idx) // 5 :]
    else:
        val_idx = train_idx

    # create embeddings
    embed_layer = RelGraphEmbed(g, args.n_hidden)

    if not args.data_cpu:
        labels = labels.to(device)
        embed_layer = embed_layer.to(device)

    if args.num_workers <= 0:
        raise ValueError(
            "The '--num_workers' parameter value is expected "
            "to be >0, but got {}.".format(args.num_workers)
        )

    node_embed = embed_layer()
    # create model
    model = EntityClassify(
        g,
        args.n_hidden,
        num_classes,
        num_bases=args.n_bases,
        num_hidden_layers=args.n_layers - 2,
        dropout=args.dropout,
        use_self_loop=args.use_self_loop,
    )

    if use_cuda:
        model.cuda()

    # train sampler
    sampler = dgl.dataloading.MultiLayerNeighborSampler(
        [args.fanout] * args.n_layers
    )
    loader = dgl.dataloading.DataLoader(
        g,
        {category: train_idx},
        sampler,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
    )

    # validation sampler
    # we do not use full neighbor to save computation resources
    val_sampler = dgl.dataloading.MultiLayerNeighborSampler(
        [args.fanout] * args.n_layers
    )
    val_loader = dgl.dataloading.DataLoader(
        g,
        {category: val_idx},
        val_sampler,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
    )

    # optimizer
    all_params = itertools.chain(model.parameters(), embed_layer.parameters())
    optimizer = th.optim.Adam(all_params, lr=args.lr, weight_decay=args.l2norm)

    # training loop
    print("start training...")
    mean = 0
    for epoch in range(args.n_epochs):
        model.train()
        optimizer.zero_grad()
        if epoch > 3:
            t0 = time.time()

        with loader.enable_cpu_affinity():
            for i, (input_nodes, seeds, blocks) in enumerate(loader):
                blocks = [blk.to(device) for blk in blocks]
                seeds = seeds[
                    category
                ]  # we only predict the nodes with type "category"
                batch_tic = time.time()
                emb = extract_embed(node_embed, input_nodes)
                lbl = labels[seeds]
                if use_cuda:
                    emb = {k: e.cuda() for k, e in emb.items()}
                    lbl = lbl.cuda()
                logits = model(emb, blocks)[category]
                loss = F.cross_entropy(logits, lbl)
                loss.backward()
                optimizer.step()

                train_acc = th.sum(logits.argmax(dim=1) == lbl).item() / len(
                    seeds
                )
                print(
                    f"Epoch {epoch:05d} | Batch {i:03d} | Train Acc: "
                    "{train_acc:.4f} | Train Loss: {loss.item():.4f} | Time: "
                    "{time.time() - batch_tic:.4f}"
                )

        if epoch > 3:
            mean = (mean * (epoch - 3) + (time.time() - t0)) / (epoch - 2)

            val_loss, val_acc = evaluate(
                model, val_loader, node_embed, labels, category, device
            )
            print(
                f"Epoch {epoch:05d} | Valid Acc: {val_acc:.4f} | Valid loss: "
                "{val_loss:.4f} | Time: {mean:.4f}"
            )
    print()
    if args.model_path is not None:
        th.save(model.state_dict(), args.model_path)

    output = model.inference(
        g,
        args.batch_size,
        "cuda" if use_cuda else "cpu",
        args.num_workers,
        node_embed,
    )
    test_pred = output[category][test_idx]
    test_labels = labels[test_idx].to(test_pred.device)
    test_acc = (test_pred.argmax(1) == test_labels).float().mean()
    print("Test Acc: {:.4f}".format(test_acc))
    print()

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="RGCN")
    parser.add_argument(
        "--dropout", type=float, default=0, help="dropout probability"
    )
    parser.add_argument(
        "--n-hidden", type=int, default=16, help="number of hidden units"
    )
    parser.add_argument("--gpu", type=int, default=-1, help="gpu")
    parser.add_argument("--lr", type=float, default=1e-2, help="learning rate")
    parser.add_argument(
        "--n-bases",
        type=int,
        default=-1,
        help="number of filter weight matrices, default: -1 [use all]",
    )
    parser.add_argument(
        "--n-layers", type=int, default=2, help="number of propagation rounds"
    )
    parser.add_argument(
        "-e",
        "--n-epochs",
        type=int,
        default=20,
        help="number of training epochs",
    )
    parser.add_argument(
        "-d", "--dataset", type=str, required=True, help="dataset to use"
    )
    parser.add_argument(
        "--model_path", type=str, default=None, help="path for save the model"
    )
    parser.add_argument("--l2norm", type=float, default=0, help="l2 norm coef")
    parser.add_argument(
        "--use-self-loop",
        default=False,
        action="store_true",
        help="include self feature as a special relation",
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        default=100,
        help="Mini-batch size. If -1, use full graph training.",
    )
    parser.add_argument(
        "--fanout", type=int, default=4, help="Fan-out of neighbor sampling."
    )
    parser.add_argument(
        "--data-cpu",
        action="store_true",
        help="By default the script puts all node features and labels "
        "on GPU when using it to save time for data copy. This may "
        "be undesired if they cannot fit in GPU memory at once. "
        "This flag disables that.",
    )
    parser.add_argument(
        "--num_workers", type=int, default=4, help="Number of node dataloader"
    )

    fp = parser.add_mutually_exclusive_group(required=False)
    fp.add_argument("--validation", dest="validation", action="store_true")
    fp.add_argument("--testing", dest="validation", action="store_false")
    parser.set_defaults(validation=True)

    args = parser.parse_args()
    print(args)
    main(args)